# Traces for draft-lake-authz

Requirements:

```python
pip install cryptography==3.4.7 cbor2==5.3.0 rich==10.6.0 hkdf==0.0.3
```

In [1]:
import rich, cbor2, hkdf, hashlib
from cryptography.hazmat.primitives import asymmetric, serialization
from cryptography.hazmat.primitives.ciphers import aead
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from binascii import hexlify, unhexlify

### Common Functions

In [14]:
def format_tv(tv, nokeys=False):
    for k, v in tv.items():
        if k[0] == "_" or (nokeys and k in ["static_keys", "ephemeral_keys"]):
            continue
        elif type(v) == dict:
            print(f"\n# {k}")
            format_tv(v)
        elif type(v) == int:
            print(f'{k:<8} = {v}')
        else:
            print(f'{k:<8} = "{v}"')

def add_new_keys(tv):
    def as_hex(k):
        return hex(k)[2:]
    def new_keypair_dx_testvector(entity_name):
        private_key = ec.generate_private_key(ec.SECP256R1(), backend=default_backend())
        x = private_key.public_key().public_numbers().x
        y = private_key.public_key().public_numbers().y
        d = private_key.private_numbers().private_value
        return {f"{entity_name}": as_hex(d), f"G_{entity_name}": as_hex(x), f"_G_{entity_name}_y": as_hex(y)}

    tv["static_keys"] = {}
    tv["ephemeral_keys"] = {}
    for a in ["U", "V", "W"]:
        tv["static_keys"].update(new_keypair_dx_testvector(a))
    for a in ["X", "Y", "Z"]:
        tv["ephemeral_keys"].update(new_keypair_dx_testvector(a))

    return tv

keys_tv = {
    'static_keys': {
        'U': '555C89F41EA42D458F0B4D74499E1C177BA9AD910F525BACF3D64D35E8568DEC',
        'G_U': 'AC69E4299F79FAED612E37C37F99D2B3939B142A8E8E65B90FAB5001F7F2CF56',
        '_G_U_y': '9572CF756D05E8B80DF519AEF4BF43E546BCB871A8BC4B676ED548F24F4EC362',
        'V': '1DAF151B30F0F247AEB5598C1EEE8664384166BBC37F262DC6581A67486BCF3C',
        'G_V': '188B3E5A62352FFCDE66894FCDBBCB33D243A045BAA99357A72012A6AF3A33AD',
        '_G_V_y': 'C19B9BCB27EF514228016F94DE85C068AFE416B80752EDF256F2593FE367766A',
        'W': '4E5E15AB35008C15B89E91F9F329164D4AACD53D9923672CE0019F9ACD98573F',
        'G_W': 'FFA4F102134029B3B156890B88C9D9619501196574174DCB68A07DB0588E4D41',
        '_G_W_y': 'BD08125C1A5E9C4F4AA60198A9F897EB656784DE50C0FE840FE3683FC20C295C'
    },
    'ephemeral_keys': {
        'X': 'A0C71BDBA570FFD270D90BDF416C142921F214406271FCF55B8567F079B50DA0',
        'G_X': 'FF14FB42677CE9D016907F571E5E1CD4E815F098AA37084063A0C34570F6F7F5',
        '_G_X_y': '353AFA30B59B6FA90843F8BECD981CEDC0A2BD3E61421EAFE171544D9994C769',
        'Y': 'A1D1A1C084AB0D912CC7A15B7F252FABCA252FAD4CAA8E5D569C94578B52A047',
        'G_Y': '9F69C52FAE8F7EA9194022C70B238FCBF4AFFFDFFC8341EEC85BA68E2F9BB744',
        '_G_Y_y': '52CB7AAF4E56C610C91A6185B92AFF5B03E9F73E6010AEBBA72B9C4BDA269C9A',
        'Z': '644658D815CBCA8EA863090A2D498990B5C75357A729231EC3DE7DF5A7AFE49E',
        'G_Z': '6B67C90638924C4AE8472CA6FB9A90BE5F43132753346379C672972D323F7A41',
        '_G_Z_y': 'FA1EFAD24A287B1FEF04683B5B24963A107067541B2E4766088552EE11337D87'
    },
}
# keys_tv = add_new_keys(keys_tv) # uncomment to generate a new set of keys

### Crypto functions

In [3]:
def p256_ecdh(d_hex, x_hex, y_hex):
    private_key = ec.derive_private_key(int(d_hex, 16), ec.SECP256R1(), default_backend())
    # NOTE: rust uses the compressed form of the public key (without the y coordinate), but the result should be the same
    public_key = ec.EllipticCurvePublicNumbers(
        int(x_hex, 16),
        int(y_hex, 16),
        ec.SECP256R1()
    ).public_key(default_backend())
    return private_key.exchange(ec.ECDH(), public_key).hex()

def hkdf_extract(salt, ikm):
    return hkdf.hkdf_extract(unhexlify(salt), unhexlify(ikm), hash=hashlib.sha256).hex()

def hkdf_expand(prk, info, length):
    return hkdf.hkdf_expand(unhexlify(prk), info, length, hash=hashlib.sha256).hex()

def aes_ccm_encrypt_tag_8(key, iv, enc_structure, plaintext):
    return aead.AESCCM(unhexlify(key)).encrypt(unhexlify(iv), unhexlify(plaintext), unhexlify(enc_structure)).hex()


### Traces for EAD_1

See https://www.ietf.org/archive/id/draft-selander-lake-authz-03.html#name-device-enrollment-server-u-

In [24]:

def add_enc_id(tv):
    salt_fixme = "00" * 32 # FIXME rust
    g_xw = p256_ecdh(tv["ephemeral_keys"]["X"], tv["static_keys"]["G_W"], tv["static_keys"]["_G_W_y"])
    prk = hkdf_extract(salt_fixme, g_xw)
    k_1 = hkdf_expand(prk, cbor2.dumps(0)+cbor2.dumps(b'')+cbor2.dumps(32), 16) # info is (0, b'', 16) # FIXME[draft] make 'length' explicit
    iv_1 = hkdf_expand(prk, cbor2.dumps(1)+cbor2.dumps(b'')+cbor2.dumps(32), 13) # info is (1, b'', 13) # FIXME[draft] make 'length' explicit
    plaintext = cbor2.dumps(unhexlify(tv["input"]["ID_U"])).hex()
    _ss = tv["input"]["SS"].to_bytes(1, byteorder='big')
    enc_structure = cbor2.dumps(["Encrypt0", b'', cbor2.dumps(_ss)]).hex()
    enc_id = aes_ccm_encrypt_tag_8(k_1, iv_1, enc_structure, plaintext)
    tv.update({
        "enc_id": {
            "enc_id": enc_id,
            "salt_fixme": salt_fixme,
            "g_xw": g_xw,
            "prk": prk,
            "k_1": k_1,
            "iv_1": iv_1,
            "plaintext": plaintext,
            "enc_structure": enc_structure,
        }
    })

    return tv

def add_voucher_info(tv):
    voucher_info_seq = (cbor2.dumps(unhexlify(tv["input"]["LOC_W"])) + cbor2.dumps(unhexlify(tv["enc_id"]["enc_id"]))).hex()
    voucher_info = cbor2.dumps(unhexlify(voucher_info_seq)).hex()
    tv.update({
        "voucher_info": {
            "voucher_info": voucher_info,
            "voucher_info_seq": voucher_info_seq,
        }
    })
    return tv

def add_ead1(tv):
    label = "01"
    value = tv["voucher_info"]["voucher_info"]
    ead1 = label + value
    tv.update({
        "ead1": {
            "ead1": ead1,
            "label": label,
            "value": value,
        }
    })
    return tv

ead1_tv = {
    "input": {
        "LOC_W": cbor2.dumps("coap://enrollment.server").hex(), # already a tstr
        "ID_U": cbor2.dumps({4: b'\x2B'}).hex(),
        "SS": 2,
    }
}
ead1_tv.update(keys_tv) # using existing keys
# ead1_tv = add_new_keys(ead1_tv) # uncomment to generate a new set of keys

ead1_tv = add_enc_id(ead1_tv)
ead1_tv = add_voucher_info(ead1_tv)
ead1_tv = add_ead1(ead1_tv)

# rich.print(ead1_tv)
format_tv(ead1_tv)
# format_tv(ead1_tv, nokeys=True)


# input
LOC_W    = "7818636f61703a2f2f656e726f6c6c6d656e742e736572766572"
ID_U     = "a104412b"
SS       = 2

# static_keys
U        = "555C89F41EA42D458F0B4D74499E1C177BA9AD910F525BACF3D64D35E8568DEC"
G_U      = "AC69E4299F79FAED612E37C37F99D2B3939B142A8E8E65B90FAB5001F7F2CF56"
V        = "1DAF151B30F0F247AEB5598C1EEE8664384166BBC37F262DC6581A67486BCF3C"
G_V      = "188B3E5A62352FFCDE66894FCDBBCB33D243A045BAA99357A72012A6AF3A33AD"
W        = "4E5E15AB35008C15B89E91F9F329164D4AACD53D9923672CE0019F9ACD98573F"
G_W      = "FFA4F102134029B3B156890B88C9D9619501196574174DCB68A07DB0588E4D41"

# ephemeral_keys
X        = "A0C71BDBA570FFD270D90BDF416C142921F214406271FCF55B8567F079B50DA0"
G_X      = "FF14FB42677CE9D016907F571E5E1CD4E815F098AA37084063A0C34570F6F7F5"
Y        = "A1D1A1C084AB0D912CC7A15B7F252FABCA252FAD4CAA8E5D569C94578B52A047"
G_Y      = "9F69C52FAE8F7EA9194022C70B238FCBF4AFFFDFFC8341EEC85BA68E2F9BB744"
Z        = "644658D815CBCA8EA863090A2D498990B5C75357A729231EC3DE7DF5A7AFE49E"

### Unit tests

In [23]:
import unittest

class Test(unittest.TestCase):
    def test_ead_1(self):
        self.assertEqual(
            p256_ecdh(tv["ephemeral_keys"]["X"], tv["static_keys"]["G_W"], tv["static_keys"]["_G_W_y"]), 
            p256_ecdh(tv["static_keys"]["W"], tv["ephemeral_keys"]["G_X"], tv["ephemeral_keys"]["_G_X_y"]), 
        )

unittest.main(argv=[''], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
